### Auteurs : Theo Dionne et Jérôme Leblanc
(La résolution de ce problème est inspiré par le carnet *5.3_condensateur.ipynb* distribué dans le cadre du cours PHQ202)

In [1]:
# Importations nécessaires 
import numpy as np
import timeit
from numba import jit
import matplotlib.pyplot as plt
import matplotlib as mpl
import warnings

# Petites modifications esthétiques
warnings.filterwarnings('ignore') # Pour taire les avertissement du compilateur jit
mpl.rcParams['figure.dpi'] = 125 # Pour avoir une meilleur résolution

In [2]:
t = timeit.default_timer() # On commence le chronomètre

L = 10.0 # Taille du domaine physique étudié (de -L à L dans les deux directions)
d = 3.0 # Largeur totale des plaques
b = 3.0 # Demi-distance entre les plaques
N = 200 # Nombre de points sur un côté de la grille
V0 = 1.0 # Valeur du potentiel sur l'une des plaques (-V sur l'autre plaque)
a = 2*L/N # Facteur de conversion cartésien <---> points

# initialisation de la grille
V = np.zeros([N, N],float)

# sélection des éléments de la grille qui correspondent aux plaques
x1 = int(np.floor((L-b-d)/a));
x2 = int(np.floor((L-b)/a));
y_plq = int(np.floor((L)/a));
plaque1 = np.arange(int(N*(L)/a + (L-b-d)/a), int(N*(L)/a + (L-b)/a))
plaque2 = np.arange(int(N*(L)/a + (L+b)/a), int(N*(L)/a + (L+b+d)/a))


# fonction qui correspond à une partie du potentiel électrique
@jit
def coulomb(x, y, xp, yp):
    D = (np.sqrt((x-xp)**2 + (y-yp)**2)/(np.sqrt((x-(xp+(d+2*b)/a))**2 + (y-yp)**2)))
    return np.log(D)/(2*np.pi)


# Application des conditions limites sur les plaques et les bordures
@jit
def conditions_limites(phi):
    global N, V0, plaque1, plaque2, x1, x2, y_plq
    
    np.put(phi,plaque1,V0) # ici phi est traité comme un tableau 'déroulé'
    np.put(phi,plaque2,-V0)


    for x in range(x1, x2):
        rho1 = -2*phi[y_plq, x] + phi[y_plq+1, x] + phi[y_plq-1, x]
        
        # on boucle sur les côtés extérieurs, sauf les coins
        for b in range(1,N-1) :
            phi[0, b] += rho1*coulomb(b, 0, x, y_plq)
            phi[b,-1] += rho1*coulomb(N-1, b, x, y_plq)
            phi[b, 0] += rho1*coulomb(0, b, x, y_plq)
            phi[-1, b] += rho1*coulomb(b, N-1, x, y_plq)
        
        # on traite les coins séparément, pour éviter de les répéter
        phi[0, 0]  += rho1*coulomb(0, 0, x, y_plq)
        phi[-1,-1] += rho1*coulomb(N-1, N-1, x, y_plq)
        phi[0, -1] += rho1*coulomb(0, N-1, x, y_plq)
        phi[-1, 0] += rho1*coulomb(N-1, 0, x, y_plq)

        
# Mettre à jour V
@jit
def update(V):
    V1 = np.zeros([N,N],float)

    for x in range(1,N-1):
        for y in range(1,N-1):
            V1[x,y] = 0.125*(V[x-1,y-1]+V[x-1,y]+V[x-1,y+1]+V[x+1,y-1]+V[x+1,y]+V[x+1,y+1]+V[x,y-1]+V[x,y+1])
    conditions_limites(V1)
    diff = V1-V
    diff = diff*diff
    diff_scal = np.sqrt(np.sum(diff))
    return V1, diff_scal

#------------------------Boucle principale--------------------------

delta = 1.0
precision = 0.001
iter = 0

# Affichage de la progression du calcul
while delta > precision :
    V,delta = update(V)
    if(iter%100==0) : print('iteration ', iter, '    delta = ', delta)
    iter += 1
    
np.savetxt('V_data.dat',V) # écriture de la solution dans un fichier
t = timeit.default_timer() - t # arrêt du chronomètre
print('temps de calcul: ', t, ', ', iter, ' iterations') # retour du temps total de calcul

iteration  0     delta =  154.73238137954223
iteration  100     delta =  0.1895855184221373
iteration  200     delta =  0.09289905388204288
iteration  300     delta =  0.06464530457141922
iteration  400     delta =  0.05027628503862147
iteration  500     delta =  0.04110515071321274
iteration  600     delta =  0.03459943198616271
iteration  700     delta =  0.02968761164694641
iteration  800     delta =  0.025813908405725478
iteration  900     delta =  0.022658632316593486
iteration  1000     delta =  0.02002599814057124
iteration  1100     delta =  0.017789997002864072
iteration  1200     delta =  0.015865710474282674
iteration  1300     delta =  0.014193345150382904
iteration  1400     delta =  0.012729059153546324
iteration  1500     delta =  0.011439542338768127
iteration  1600     delta =  0.010298718928372972
iteration  1700     delta =  0.009285675096649034
iteration  1800     delta =  0.008383309634153464
iteration  1900     delta =  0.00757742278979127
iteration  2000     delt

In [3]:
# utilisation du backend ipympl pour le graphique
%matplotlib ipympl 

V = np.loadtxt('V_data.dat',float)  # lecture de la solution à partir d'un fichier

# Graphique et options graphiques
ax = plt.gca()
ax.set_aspect(1)

x = np.linspace(-L,L,N)
CS = plt.contour(x, -x, V, 20, colors='blue', linewidths=0.5)

Ey, Ex = np.gradient(V)
Ex *= -1
plt.streamplot(x, -x, Ex, Ey, color='red', linewidth=0.5)

plt.xlabel('$x$')
plt.ylabel('$y$')

# rajout de deux lignes pour représenter les plaques chargées
p1 = plt.Line2D((-b-d, -b), (0, 0), lw=2.5,color='black')
p2 = plt.Line2D((b, b+d), (0, 0), lw=2.5,color='black')
ax.add_line(p1)
ax.add_line(p2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …